# Homework w04d04

The attached dataset includes data about students being admitted to graduate school, their GRE (Graduate Record Exam scores), GPA (Grade Point Average) and the prestige of their undergraduate institution. The aim is to predict the probability of being admitted to graduate school using the remaining features with logistic regression.

The variables have the following ranges:

1. Admit: 1 (admitted), 0 (not admitted)
1. GRE: 0 (lowest) to 800 (highest)
1. GPA: 0 (lowest) to 4 (highest)
1. Prestige: 1 (highest) to 4 (lowest)

In [1]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab as pl
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline

# Optional
import seaborn as sns
sns.set_style("darkgrid")
from sklearn import model_selection, linear_model, metrics

# Import data
df = pd.read_csv("admissions.csv").dropna()
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part 1. Frequency Tables

#### Create a frequency table of our variables

In [2]:
# Create a frequency table for prestige vs whether or not someone was admitted (hint: look at pd.crosstab())
pd.crosstab(df.admit,df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Dummy variables

#### 2.1 Create four new dummy variables for prestige

In [3]:
# Create dummy vars here
dummy_ranks = pd.get_dummies(df.prestige,prefix='prestige',prefix_sep='_')
dummy_ranks.rename(columns={'prestige_1.0':'prestige_1','prestige_2.0':'prestige_2','prestige_3.0':'prestige_3','prestige_4.0':'prestige_4'},inplace=True)
dummy_ranks.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


#### 2.2 When modelling our prestige categorical variables, how many do we need? Why?
All 4? 3? 2? 1?

Answer: We only need three variables because we know that in each row only one of the four variables is 1 while the other ones are zero which determines the fourth variable if we know the other three.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [4]:
dummy_ranks.ix[:,'prestige1':].head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


In [5]:
cols_to_keep = ['admit', 'gre', 'gpa']
hand_calc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
hand_calc.head()

,admit,gre,gpa,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0


#### 3.1 Cross-tabulate prestige_1 admission

In [6]:
# Use pd.crosstab to create a frequency table of prestige_1 vs admission
crosstab_p1 = pd.crosstab(hand_calc.admit,hand_calc.prestige_1)
crosstab_p1

prestige_1,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.2 Use the cross-tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [7]:
admit_prob_p1 = crosstab_p1.iloc[1,1] / float(crosstab_p1.iloc[1,1] + crosstab_p1.iloc[0,1])
admit_odds_p1 = admit_prob_p1 / (1 - admit_prob_p1)
print 'probability: ', admit_prob_p1, 'odds: ', admit_odds_p1

probability:  0.540983606557 odds:  1.17857142857


#### 3.3 Now calculate the odds of admission if you did not attend a #1 ranked college

In [8]:
admit_prob_pnot1 = crosstab_p1.iloc[1,0] / float(crosstab_p1.iloc[1,0] + crosstab_p1.iloc[0,0])
admit_odds_pnot1 = admit_prob_pnot1 / (1 - admit_prob_pnot1)
print 'probability: ', admit_prob_pnot1, 'odds: ', admit_odds_pnot1

probability:  0.276785714286 odds:  0.382716049383


#### 3.4 Calculate the odds ratio

In [9]:
admit_odds_p1 / admit_odds_pnot1

3.0794930875576041

#### 3.5 Write this finding in a sentence 

Answer: The odds of being admitted to graduate school after having attended a prestige_1 school are approximately 3:1.

#### 3.6 Print the cross-tab vs prestige_4

In [10]:
crosstab_p4 = pd.crosstab(hand_calc.admit,hand_calc.prestige_4)
crosstab_p4

prestige_4,0.0,1.0
admit,,
0,216,55
1,114,12


#### 3.7 Calculate the odds ratio

In [11]:
admit_prob_p4 = crosstab_p4.iloc[1,1] / float(crosstab_p4.iloc[1,1] + crosstab_p4.iloc[0,1])
admit_odds_p4 = admit_prob_p4 / (1 - admit_prob_p4)
print 'probability: ', admit_prob_p4, 'odds: ', admit_odds_p4

admit_prob_pnot4 = crosstab_p4.iloc[1,0] / float(crosstab_p4.iloc[1,0] + crosstab_p4.iloc[0,0])
admit_odds_pnot4 = admit_prob_pnot4 / (1 - admit_prob_pnot4)
print 'probability: ', admit_prob_pnot4, 'odds: ', admit_odds_pnot4
print 'odds ratio: ', admit_odds_p4 / admit_odds_pnot4

probability:  0.179104477612 odds:  0.218181818182
probability:  0.345454545455 odds:  0.527777777778
odds ratio:  0.413397129187


#### 3.8 Write this finding in a sentence

Answer: The odds of being admitted to graduate school after having attended a prestige_4 school are approximately 2:5.

## Part 4. Analysis
First we'll create a clean data frame for the regression analysis.

In [12]:
# We'll set the top tier (#1, aka most prestigious) as our reference category
# and merge prestige_2, prestige_3 and prestige_4 back into the dataset
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,0.0,1.0,0.0
1,1,660.0,3.67,0.0,1.0,0.0
2,1,800.0,4.00,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0


We're going to add a constant term for our logistic regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [13]:
# Manually add the intercept
data['intercept'] = 1.0
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4,intercept
0,0,380.0,3.61,0.0,1.0,0.0,1.0
1,1,660.0,3.67,0.0,1.0,0.0,1.0
2,1,800.0,4.00,0.0,0.0,0.0,1.0
3,1,640.0,3.19,0.0,0.0,1.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0,1.0


#### 4.1 Assign the predictor column names to a variable called train_cols

In [14]:
train_cols = data.columns[1:]
print train_cols

Index([u'gre', u'gpa', u'prestige_2', u'prestige_3', u'prestige_4',
       u'intercept'],
      dtype='object')


#### 4.2 Fit a logistic regression model of using statsmodels
Want to model admin ~ gre + gpa + presige_2 + prestige_3 + prestige_4 + intercept using logistic regression.

In [15]:
logit = sm.Logit(data.admit,data[train_cols])

#### 4.3 Print the logistic regression summary results

In [23]:
lr_form = smf.logit(formula = 'admit ~ gre + gpa + prestige_2 + prestige_3 + prestige_4', data = data).fit()
#lr_form_result = lr_form.fit()
lr_form.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 04 Nov 2016   Pseudo R-squ.:                 0.08166
Time:                        11:41:34   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
==============================================================================
"""

In [17]:
logit = sm.Logit(data.admit,data[train_cols])
lr_result = logit.fit()
lr_result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 04 Nov 2016   Pseudo R-squ.:                 0.08166
Time:                        11:40:05   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2    -0.6801      0.317     -2.146      0.032        -1.301    -0.059
prestige_3    -1.3387      0.345     -3.882      0.000        -2.015    -0.663
prestige_4    -1.5534      0.417     -3.721      0.000        -2.372    -0.735
intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
==============================================================================
"""

In [18]:
# Scikit-learn implementation - default 
model = linear_model.LogisticRegression(fit_intercept=False)
model.fit(data[train_cols],data.admit)
print model.coef_

[[  1.78497168e-03   2.32294580e-01  -6.03474675e-01  -1.17214957e+00
   -1.37729795e+00  -1.81701706e+00]]


In [19]:
# Scikit-learn implementation - suppressing regularisation
model = linear_model.LogisticRegression(C=1e30,fit_intercept=False)
model.fit(data[train_cols],data.admit)
print model.coef_

[[  2.14202407e-03   6.76070461e-01  -6.31200468e-01  -1.25232414e+00
   -1.56000526e+00  -3.51478098e+00]]


In [20]:
# Scikit-learn implementation suppressing regularisation and same solve as statsmodels
model = linear_model.LogisticRegression(C=1e30,fit_intercept=False,solver='newton-cg')
model.fit(data[train_cols],data.admit)
print model.coef_

[[  2.21840209e-03   7.79336465e-01  -6.80136377e-01  -1.33867670e+00
   -1.55341002e+00  -3.87685096e+00]]


In [21]:
(np.array(lr_result.params) - model.coef_) / model.coef_

array([[  5.47878809e-07,   9.80041555e-07,   3.33920876e-07,
          3.75033129e-07,   3.13151414e-07,   8.03488323e-07]])

#### 4.4  Give a short interpretation of the regression coefficients

Lower prestige has a negative influence on admission, gpa and gre have a positive influence. The small coefficient of gre reflects the different range.